In [1]:
import pandas as pd

In [2]:
df_raw = pd.read_csv(r'M:\Data\OnBoard\Data and Reports\Tri Delta\2019\TriDelta_ODSurvey_Dataset_Weights_03272019_FinalDeliv.csv')

In [3]:
df = df_raw.copy()

In [4]:
df['Final_ORIGIN_TRANSPORT_combine'] = df['Final_ORIGIN_TRANSPORT']
df.loc[df.Final_ORIGIN_TRANSPORT == 'Other', 'Final_ORIGIN_TRANSPORT_combine'] = df['Final_ORIGIN_TRANSPORT_other']

df['final_Destin_place_type_combine'] = df['final_Destin_place_type']
df.loc[df.final_Destin_place_type == 'Other', 'final_Destin_place_type_combine'] = df['final_Destin_place_type_other']

print(df.shape[0])
print(list(df))

df.to_csv(r'M:\Data\OnBoard\Data and Reports\Tri Delta\2019\TriDelta_ODSurvey_Dataset_Weights_03272019_FinalDeliv_addCols_NO POUND OR SINGLE QUOTE.csv', index=False)

742
['ID', 'SURVEY_TYPE', 'DOW', 'DATE', 'ROUTE_SURVEYED_CODE', 'ROUTE_SURVEYED', 'change_to_route_surveyed', 'reason_for_change_to_route_surveyed', 'final_route_surveyed_code', 'final_route', 'RESIDENT_OR_VISITOR_Code', 'RESIDENT_OR_VISITOR', 'HOME_ZIP', 'HOME_ADDRESS_ADDR', 'HOME_ADDRESS_CITY', 'HOME_ADDRESS_STATE', 'HOME_ADDRESS_ZIP', 'HOME_ADDRESS_LAT', 'HOME_ADDRESS_LONG', 'HOTEL_ADDRESS_ADDR', 'HOTEL_ADDRESS_CITY', 'HOTEL_ADDRESS_STATE', 'HOTEL_ADDRESS_ZIP', 'HOTEL_ADDRESS_LAT', 'HOTEL_ADDRESS_LONG', 'origin_place_type_code', 'ORIGIN_PLACE_TYPE', 'ORIGIN_PLACE_TYPE_Other', 'change_to_origin_place_type', 'reason_for_change_to_origin_place_type', 'updated_origin_place_type_code', 'updated_origin_place_type', 'updated_origin_place_type_other', 'final_change_to_origin_place_type', 'final_reason_for_change_to_origin_place_type', 'final_origin_place_type_code', 'final_origin_place_type', 'ORIGIN_ADDRESS_ADDR', 'ORIGIN_ADDRESS_CITY', 'ORIGIN_ADDRESS_STATE', 'ORIGIN_ADDRESS_ZIP', 'ORIGIN

In [5]:
routes = pd.DataFrame(columns = ['survey_name'])
for i in ['Final_TRIP_FIRST_ROUTE', 'Final_TRIP_SECOND_ROUTE',
          'Final_TRIP_NEXT_ROUTE', 'Final_TRIP_AFTER_ROUTE', 
          'Final_TRIP_3RD_ROUTE']:
    route_unique = df[[i]]
    route_unique.columns = ['survey_name']
    routes = pd.concat([routes, route_unique])

routes_clean = routes.loc[routes.survey_name.notnull()]
routes_clean.drop_duplicates(inplace=True)
routes_clean['survey'] = 'TriDelta'
routes_clean['survey_year'] = 2019

print(routes_clean.shape)
routes_clean[['survey','survey_year','survey_name']].to_csv(r'M:\Data\OnBoard\Data and Reports\Tri Delta\2019\all_routes_raw.csv', index=False)

(53, 3)


C:\Users\ywang\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\ywang\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\ywang\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

In [6]:
# bring in standard dictionary to check field consistency

# dictionary for TriDelta survey
var = pd.read_csv(r'M:\Data\OnBoard\Data and Reports\Tri Delta\2019\variable_dictionary.csv',
                  encoding = "ISO-8859-1", engine='python')

# standard dictionary
var_standard = pd.read_csv(r'C:\Users\ywang\Documents\GitHub\onboard-surveys\make-uniform\production\Dictionary for Standard Database.csv')
var_standard.columns = [x+'_s' for x in list(var_standard)]

# merge
var_merge = var.merge(var_standard, left_on='Generic_Variable', right_on='Generic_Variable_s', how='outer')

# check if 'Generic_Variable' in TriDelta dictionary matches the standard 'Generic_Variable'. chk1 should be empty
chk1 = var_merge.loc[(var_merge.Generic_Variable.notnull()) & (var_merge.Generic_Variable_s.isnull())]
print('Generic_Variable that should not exit:')
print(chk1.Generic_Variable.unique())
print()

# check if columns names in survey data matches 'Survey_Variable' in TriDelta dictionary.
# the following loops should not include variables that are needed for standardization

for i in var.loc[var.Generic_Variable.notnull()]['Survey_Variable']:
    if i not in list(df):
        print(i)
        
for i in list(df):
    if i not in list(var.Survey_Variable):
        print(i)

Generic_Variable that should not exit:
['time_period']



In [7]:
# check if all the values in the survey data are included in TriDelta dictionary
# look at non-categorical variables; "diff" should be empty or only contains nan

var_clean = var[['operator', 'Survey_year', 'Survey_Variable', 'Survey_Response', 
                 'Generic_Variable', 'Generic_Response']].drop_duplicates()
var_clean = var_clean.loc[var_clean.Generic_Variable.notnull()]

for i in var_clean.loc[var_clean.Survey_Response != 'NONCATEGORICAL']['Survey_Variable'].unique():
    print(i)
    df_sub = df[['ID', i]]
    var_sub = var_clean.loc[var_clean.Survey_Variable == i]
    
    compare = df_sub.merge(var_sub, left_on=i, right_on='Survey_Response', how='left')
    diff = compare.loc[compare.Generic_Response.isnull()]
    if diff.shape[0] > 0:
        print(diff.Survey_Response.unique())

final_origin_place_type
[nan]
Final_ORIGIN_TRANSPORT_combine
final_Destin_place_type_combine
final_DESTIN_TRANSPORT
TIME_LEFT_HOME
TIME_ARRIVE_HOME
TIME_ON
TIME_PERIOD
FARE_CATEGORY
KIND_OF_FARE
COUNT_VH_HH
HH_SIZE
[nan]
Final_EMPLOYED_HH_MEMBERS
[nan]
STATUS_EMPLOYMENT
[nan]
WORKED_BEFORE_TRIP
[nan]
WORK_AFTER_TRIP
[nan]
STUDENT_STATUS
WENT_TO_SCHL
[nan]
GOING_TO_SCHL
[nan]
ETHNIC_BACKGROUND_1
ETHNIC_BACKGROUND_2
ETHNIC_BACKGROUND_3
ETHNIC_BACKGROUND_6
ETHNIC_BACKGROUND_5
ETHNIC_BACKGROUND_8
GENDER
INCOME
HOME_LANG_OTHER
HOME_OTHER_LANG
[nan]
ENGLISH_ABILITY
[nan]


In [8]:
# finally, check all necessary fields are included, and export
print(var_clean.Generic_Variable.unique())
var_clean.to_csv(r'M:\Data\OnBoard\Data and Reports\Tri Delta\2019\vars_for_standard_dictionary.csv', index=False)

['ID' 'date_string' 'route' 'home_lat' 'home_lon' 'orig_purp' 'orig_lat'
 'orig_lon' 'number_transfers_orig_board'
 'first_route_before_survey_board' 'second_route_before_survey_board'
 'access_mode' 'dest_purp' 'dest_lat' 'dest_lon'
 'number_transfers_alight_dest' 'first_route_after_survey_alight'
 'second_route_after_survey_alight' 'third_route_after_survey_alight'
 'egress_mode' 'survey_board_lat' 'survey_board_lon' 'survey_alight_lat'
 'survey_alight_lon' 'first_board_lat' 'first_board_lon' 'last_alight_lat'
 'last_alight_lon' 'depart_hour' 'return_hour' 'time_string' 'time_period'
 'fare_category' 'fare_medium' 'vehicles' 'persons' 'workers'
 'work_status' 'workplace_lat' 'workplace_lon'
 'at_work_prior_to_orig_purp' 'at_work_after_dest_purp' 'student_status'
 'school_lat' 'school_lon' 'at_school_prior_to_orig_purp'
 'at_school_after_dest_purp' 'year_born_four_digit' 'hispanic'
 'race_dmy_blk' 'race_dmy_asn' 'race_dmy_ind' 'race_dmy_hwi'
 'race_dmy_wht' 'race_other_string' 'gender